# Modify an NDEx Network using NiceCX

First, we import the ndex2 Python client and the NiceCX components

In [12]:
from nicecxModel.NiceCXNetwork import NiceCXNetwork
from nicecxModel.cx.aspects.NodesElement import NodesElement
from nicecxModel.cx.aspects.EdgesElement import EdgesElement
from nicecxModel.cx.aspects.NodeAttributesElement import NodeAttributesElement
from nicecxModel.cx.aspects.EdgeAttributesElement import EdgeAttributesElement
import ndex2
import json
import DecimalEncoder

Next, we set a variable to hold the UUID of the network we want to modify

In [13]:
my_network_uuid = 'f1dd6cc3-0007-11e6-b550-06603eb7f303'

We set variables to hold the user's account and password 

In [ ]:
my_account = "my_account"
my_password = "my_password"

Now we get the network from NDEx an make a NiceCX object

In [27]:
my_network = ndex2.create_nice_cx_from_server(server='public.ndexbio.org', uuid=my_network_uuid, 
                                              account=my_account, password = my_password)
print(my_network.getSummary())
print(my_network)

http://public.ndexbio.org/v2/network/f1dd6cc3-0007-11e6-b550-06603eb7f303/aspect
Name: CoCaNet2
Nodes: 36
Edges: 37
Node Attributes: 778
Edge Attributes: 659

[{"numberVerification": [{"longNumber": 281474976710655}]}, {"metaData": [{"elementCount": 36, "version": "1.0", "consistencyGroup": 1, "name": "nodes", "idCounter": 212}, {"elementCount": 37, "version": "1.0", "consistencyGroup": 1, "name": "edges", "idCounter": 212}, {"elementCount": 11, "consistencyGroup": 1, "name": "networkAttributes"}, {"elementCount": 778, "consistencyGroup": 1, "name": "nodeAttributes"}, {"elementCount": 659, "consistencyGroup": 1, "name": "edgeAttributes"}, {"elementCount": 36, "consistencyGroup": 1, "name": "cartesianLayout"}, {"elementCount": 6, "consistencyGroup": 1, "name": "visualProperties"}, {"elementCount": 1, "consistencyGroup": 1, "name": "subNetworks"}, {"elementCount": 1, "consistencyGroup": 1, "name": "cyViews"}, {"elementCount": 2, "consistencyGroup": 1, "name": "networkRelations"}, {"eleme

We want to merge two node attributes into one. We therefore:

* Iterate over all the nodes in the network and for each one we check the values of the two attributes. 
* Then we add the new attribute based on the first two. 
* Finally, we delete the two original attributes

In [35]:
for k,v in my_network.getEdgeAttributes().items():
    for att in v:
        print(att)
    
for k,v in my_network.getNodeAttributes().items():
    print(v)
#print(my_network.getEdges())

{"po": 134, "n": "known interaction", "d": "string", "s": 52, "v": ""}
{"po": 134, "n": "Product", "d": "string", "s": 52, "v": "3.286424129"}
{"po": 134, "n": "yeastSscore", "d": "string", "s": 52, "v": "-5.011"}
{"po": 134, "n": "T-stat", "d": "string", "s": 52, "v": "-2.595"}
{"po": 134, "n": "IsBidirectional", "d": "string", "s": 52, "v": ""}
{"po": 134, "n": "yeast2", "d": "string", "s": 52, "v": "YBL016W"}
{"po": 134, "n": "HumanRanks", "d": "string", "s": 52, "v": "1425"}
{"po": 134, "n": "yeast1", "d": "string", "s": 52, "v": "YOR236W"}
{"po": 134, "n": "shared name", "d": "string", "s": 52, "v": "DHFR (pp) MAPK1"}
{"po": 134, "n": "druggable", "d": "string", "s": 52, "v": "DHFR"}
{"po": 134, "n": "name", "d": "string", "s": 52, "v": "DHFR (pp) MAPK1"}
{"po": 134, "n": "interaction", "d": "string", "s": 52, "v": "pp"}
{"po": 134, "n": "prior report of SL", "d": "string", "s": 52, "v": "no"}
{"po": 134, "n": "YeastRanks", "d": "string", "s": 52, "v": "1694"}
{"po": 134, "n": "hu

To check if we are getting what we want, we will print out a sample of the nodes in the modified network

In [24]:
# print(json.dumps(my_network.to_json(), sort_keys=True, indent=4))
json.dumps(my_network.to_json(),cls=DecimalEncoder)

NameError: name 'DecimalEncoder' is not defined

In [ ]:
Finally, we upload the network in NDEx, either creating a new network or updating the orignial network. 

You can un-comment the appropriate line of code below to choose how it is uploaded.  

In [ ]:
upload_message = "no upload function chosen"
# upload_message = my_network.update_to(my_network_uuid, my_server, my_account, my_password)
# upload_message = my_network.upload_to(my_server, my_account, my_password)
print(upload_message)